# Граф знаний с интерфейсом Gradio

Этот ноутбук создает интерактивный интерфейс для работы с графом знаний на основе статей из Википедии.


## Установка необходимых библиотек


In [ ]:
%pip install -q llama-index llama-index-readers-wikipedia llama-index-embeddings-huggingface llama-index-llms-huggingface gradio transformers torch peft accelerate bitsandbytes sentence-transformers langchain-huggingface pyvis


## Импорт библиотек


In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.readers.wikipedia import WikipediaReader
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch
import gradio as gr
from typing import Optional

## Настройка модели и токенизатора

Используем русскоязычную модель от SberDevice. В данном случае используем модель, которая хорошо работает с русским языком.


In [3]:
# Настройка HuggingFace токена (если требуется)
import os
os.environ["HF_TOKEN"] = ""

# Используем русскоязычную модель от SberDevice
MODEL_NAME = "ai-forever/rugpt3large_based_on_gpt2"
print(f"Используемая модель: {MODEL_NAME}")

Используемая модель: ai-forever/rugpt3large_based_on_gpt2


## Вспомогательные функции для работы с моделью


In [ ]:
%pip install wikipedia

In [5]:
def messages_to_prompt(messages):
    """Преобразование сообщений в промпт для модели"""
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # Убеждаемся, что начинаем с системного промпта
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # Добавляем финальный промпт ассистента
    prompt = prompt + "<s>bot\n"
    return prompt

def completion_to_prompt(completion):
    """Преобразование запроса в промпт"""
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"

## Инициализация глобальных переменных


In [ ]:
# Глобальные переменные для хранения индекса и модели
indexKG = None
llm = None
embed_model = None
storage_context = None

def initialize_model():
    """Инициализация модели и эмбеддингов"""
    global llm, embed_model

    # Настройка параметров квантования для экономии памяти
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    try:
        # Попытка загрузить модель с PEFT (LoRA)
        config = PeftConfig.from_pretrained(MODEL_NAME)
        base_model = AutoModelForCausalLM.from_pretrained(
            config.base_model_name_or_path,
            quantization_config=quantization_config,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        model = PeftModel.from_pretrained(base_model, MODEL_NAME, torch_dtype=torch.float16)
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

        # Пытаемся загрузить конфиг генерации
        try:
            generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
        except:
            generation_config = GenerationConfig(
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )
    except Exception as e:
        print(f"PEFT не доступен, загружаем обычную модель. Ошибка: {e}")
        # Если PEFT не доступен, загружаем обычную модель
        try:
            model = AutoModelForCausalLM.from_pretrained(
                MODEL_NAME,
                quantization_config=quantization_config,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
            generation_config = GenerationConfig(
                max_new_tokens=512,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )
        except Exception as e2:
            return f"Ошибка загрузки модели: {e2}"

    model.eval()

    # Настройка pad_token если его нет
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Создание LLM для LlamaIndex
        llm = HuggingFaceLLM(
            model=model,
            model_name=MODEL_NAME,
            tokenizer=tokenizer,
            max_new_tokens=generation_config.max_new_tokens if hasattr(generation_config, 'max_new_tokens') else 512,
            model_kwargs={"quantization_config": quantization_config},
            generate_kwargs={
                "temperature": generation_config.temperature if hasattr(generation_config, 'temperature') else 0.7,
                "do_sample": True,
                "top_p": generation_config.top_p if hasattr(generation_config, 'top_p') else 0.9,
            },
            messages_to_prompt=messages_to_prompt,
            completion_to_prompt=completion_to_prompt,
            device_map="auto",
        )
    except Exception as e:
        # Если специальный формат промпта не работает, используем стандартный
        print(f"Используем стандартный формат промпта. Ошибка: {e}")
        llm = HuggingFaceLLM(
            model=model,
            model_name=MODEL_NAME,
            tokenizer=tokenizer,
            max_new_tokens=512,
            model_kwargs={"quantization_config": quantization_config},
            generate_kwargs={
                "temperature": 0.7,
                "do_sample": True,
                "top_p": 0.9,
            },
            device_map="auto",
        )

    # Настройка модели эмбеддингов
    embed_model = HuggingFaceEmbedding(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )

    # Настройка Settings
    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.chunk_size = 512

    return "Модель успешно инициализирована!"

# Инициализируем модель при загрузке
print("Начинаем инициализацию модели...")
init_status = initialize_model()
print(init_status)

## Функции для работы с графом знаний


In [7]:
def parse_wikipedia_and_build_graph(article_titles: str) -> str:
    """
    Парсит статьи из Википедии и строит граф знаний

    Args:
        article_titles: Названия статей через запятую

    Returns:
        Сообщение о статусе построения графа
    """
    global indexKG, storage_context

    if llm is None or embed_model is None:
        return "Ошибка: Модель не инициализирована. Пожалуйста, сначала инициализируйте модель."

    try:
        # Разделяем названия статей
        titles = [title.strip() for title in article_titles.split(',') if title.strip()]

        if not titles:
            return "Ошибка: Не указаны названия статей."

        # Загрузка страниц из википедии
        reader = WikipediaReader()
        docs = reader.load_data(
            pages=titles,
            lang_prefix='ru'
        )

        if not docs:
            return f"Ошибка: Не удалось загрузить статьи: {', '.join(titles)}"

        # Создаем графовое хранилище
        graph_store = SimpleGraphStore()
        storage_context = StorageContext.from_defaults(graph_store=graph_store)

        # Создаем индекс графа знаний
        indexKG = KnowledgeGraphIndex.from_documents(
            documents=docs,
            max_triplets_per_chunk=3,
            show_progress=True,
            include_embeddings=True,
            storage_context=storage_context
        )

        return f"Граф знаний успешно построен для статей: {', '.join(titles)}\nКоличество документов: {len(docs)}"

    except Exception as e:
        return f"Ошибка при построении графа знаний: {str(e)}"

In [8]:
def query_knowledge_graph(query: str) -> str:
    """
    Выполняет запрос к графу знаний

    Args:
        query: Вопрос пользователя

    Returns:
        Ответ модели на основе графа знаний
    """
    global indexKG

    if indexKG is None:
        return "Ошибка: Граф знаний не построен. Пожалуйста, сначала распарсите данные из Википедии."

    if not query or not query.strip():
        return "Ошибка: Введите вопрос."

    try:
        # Создаем движок запросов
        query_engine = indexKG.as_query_engine(include_text=True, verbose=True)

        # Формируем промпт
        message_template = f"""<s>system
Отвечай в соответствии с Источником. Проверь, есть ли в Источнике упоминания о ключевых словах Вопроса.
Если нет, то просто скажи: 'я не знаю'. Не придумывай!</s>
<s>user
Вопрос: {query}
Источник:
</s>
"""

        # Выполняем запрос
        response = query_engine.query(message_template)

        return f"Ответ:\n{response.response}"

    except Exception as e:
        return f"Ошибка при выполнении запроса: {str(e)}"


## Создание интерфейса Gradio


In [ ]:
def process_and_query(article_titles: str, query: str) -> tuple:
    """
    Комбинированная функция: парсит данные и выполняет запрос

    Args:
        article_titles: Названия статей через запятую
        query: Вопрос пользователя

    Returns:
        Кортеж (статус парсинга, ответ на вопрос)
    """
    # Сначала парсим данные
    parse_status = parse_wikipedia_and_build_graph(article_titles)

    # Если парсинг успешен, выполняем запрос
    if "успешно построен" in parse_status.lower():
        answer = query_knowledge_graph(query)
    else:
        answer = "Сначала необходимо успешно распарсить данные."

    return parse_status, answer

# Создание интерфейса Gradio
with gr.Blocks(title="Граф знаний на основе Википедии") as demo:
    gr.Markdown("# Граф знаний с интерфейсом Gradio")
    gr.Markdown("Введите названия статей из Википедии (через запятую) и вопрос для поиска в графе знаний.")

    with gr.Row():
        with gr.Column():
            article_input = gr.Textbox(
                label="Названия статей из Википедии",
                placeholder="Например: Искусственный интеллект, Машинное обучение",
                lines=2
            )
            parse_btn = gr.Button("Распарсить данные", variant="primary")
            parse_status = gr.Textbox(
                label="Статус парсинга",
                lines=3,
                interactive=False
            )

        with gr.Column():
            query_input = gr.Textbox(
                label="Ваш вопрос",
                placeholder="Например: Что такое искусственный интеллект?",
                lines=2
            )
            query_btn = gr.Button("Отправить запрос", variant="primary")
            answer_output = gr.Textbox(
                label="Ответ",
                lines=10,
                interactive=False
            )

    # Отдельная кнопка для комбинированного действия
    gr.Markdown("---")
    gr.Markdown("### Или выполните оба действия сразу:")
    combined_btn = gr.Button("Распарсить данные и отправить запрос", variant="secondary")

    # Обработчики событий
    parse_btn.click(
        fn=parse_wikipedia_and_build_graph,
        inputs=article_input,
        outputs=parse_status
    )

    query_btn.click(
        fn=query_knowledge_graph,
        inputs=query_input,
        outputs=answer_output
    )

    combined_btn.click(
        fn=process_and_query,
        inputs=[article_input, query_input],
        outputs=[parse_status, answer_output]
    )

# Запуск интерфейса
demo.launch(share=True)

## Примечания и инструкции

### Использование интерфейса:

1. **Инициализация модели**: Модель инициализируется автоматически при запуске ячейки. Это может занять некоторое время.

2. **Парсинг данных**:
   - Введите названия статей из Википедии через запятую (например: "Искусственный интеллект, Машинное обучение")
   - Нажмите кнопку "Распарсить данные"
   - Дождитесь сообщения об успешном построении графа знаний

3. **Отправка запроса**:
   - Введите ваш вопрос в поле "Ваш вопрос"
   - Нажмите кнопку "Отправить запрос"
   - Ответ появится в поле "Ответ"

4. **Комбинированное действие**: Можно использовать кнопку "Распарсить данные и отправить запрос" для выполнения обоих действий сразу.

### Альтернативные модели:

Если модель `ai-forever/rugpt3large_based_on_gpt2` не работает, можно попробовать:
- `sberbank-ai/rugpt3large_based_on_gpt2`
- `IlyaGusev/saiga_mistral_7b` (требует токен HuggingFace)
- Другие русскоязычные модели на HuggingFace

### Требования:

- Интернет-соединение для загрузки моделей и статей из Википедии
- Достаточно памяти для загрузки модели (рекомендуется GPU)
- Токен HuggingFace может потребоваться для некоторых моделей
